In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import datadotworld as ddw
import re
import json
import plotly.express as px
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

data loading (3 csv files found on kaggle)

In [5]:
df_avg_household_size = pd.read_csv("avg-household-size.csv")
df_cancer_reg = pd.read_csv("cancer_reg.csv")
df_life_expectancy = pd.read_csv("Life Expectancy Data.csv")

In [6]:
df_avg_household_size.head(), df_cancer_reg.head(), df_life_expectancy.head()

(   statefips  countyfips  avghouseholdsize                           geography
 0          2          13              2.43      Aleutians East Borough, Alaska
 1          2          16              3.59  Aleutians West Census Area, Alaska
 2          2          20              2.77      Anchorage Municipality, Alaska
 3          2          50              3.86          Bethel Census Area, Alaska
 4          2          60              2.50         Bristol Bay Borough, Alaska,
    avganncount  avgdeathsperyear  target_deathrate  incidencerate  medincome  popest2015  povertypercent  studypercap           binnedinc  medianage  medianagemale  medianagefemale                     geography  percentmarried  pctnohs18_24  pcths18_24  pctsomecol18_24  pctbachdeg18_24  pcths25_over  pctbachdeg25_over  pctemployed16_over  pctunemployed16_over  pctprivatecoverage  pctprivatecoveragealone  pctempprivcoverage  pctpubliccoverage  pctpubliccoveragealone   pctwhite  pctblack  pctasian  pctotherrace  pc

In [ ]:

dataset_info = {
    "df_avg_household_size": {
        "Rows": df_avg_household_size.shape[0],
        "Columns": df_avg_household_size.shape[1],
        "Column Names": df_avg_household_size.columns.tolist(),
    },
    "df_cancer_reg": {
        "Rows": df_cancer_reg.shape[0],
        "Columns": df_cancer_reg.shape[1],
        "Column Names": df_cancer_reg.columns.tolist(),
    },
    "df_life_expectancy": {
        "Rows": df_life_expectancy.shape[0],
        "Columns": df_life_expectancy.shape[1],
        "Column Names": df_life_expectancy.columns.tolist(),
    },
}


In [10]:
dataset_info


{'df_avg_household_size': {'Rows': 3220,
  'Columns': 4,
  'Column Names': ['statefips',
   'countyfips',
   'avghouseholdsize',
   'geography']},
 'df_cancer_reg': {'Rows': 3047,
  'Columns': 33,
  'Column Names': ['avganncount',
   'avgdeathsperyear',
   'target_deathrate',
   'incidencerate',
   'medincome',
   'popest2015',
   'povertypercent',
   'studypercap',
   'binnedinc',
   'medianage',
   'medianagemale',
   'medianagefemale',
   'geography',
   'percentmarried',
   'pctnohs18_24',
   'pcths18_24',
   'pctsomecol18_24',
   'pctbachdeg18_24',
   'pcths25_over',
   'pctbachdeg25_over',
   'pctemployed16_over',
   'pctunemployed16_over',
   'pctprivatecoverage',
   'pctprivatecoveragealone',
   'pctempprivcoverage',
   'pctpubliccoverage',
   'pctpubliccoveragealone',
   'pctwhite',
   'pctblack',
   'pctasian',
   'pctotherrace',
   'pctmarriedhouseholds',
   'birthrate']},
 'df_life_expectancy': {'Rows': 2938,
  'Columns': 22,
  'Column Names': ['Country',
   'Year',
   'Sta

**DATAVIZ**

dataviz for first csv (avg-household-size) (not so relevant in our case).

In [12]:
# Visualization 1: Distribution of Average Household Size
fig1 = px.histogram(
    df_avg_household_size, 
    x="avghouseholdsize", 
    nbins=30, 
    title="Distribution of Average Household Size",
    labels={"avghouseholdsize": "Average Household Size"},
    opacity=0.75
)
fig1.show()

In [18]:
df_sorted = df_avg_household_size.sort_values(by='avghouseholdsize', ascending=False).head(20)


In [19]:
fig1 = px.bar(df_sorted, 
             x='geography', 
             y='avghouseholdsize', 
             title='Top 20 Counties by Average Household Size',
             labels={'geography': 'County', 'avghouseholdsize': 'Average Household Size'},
             color='avghouseholdsize', 
             color_continuous_scale='Blues')
fig1.show()

In [24]:
fig4 = px.box(df_avg_household_size, 
              x='statefips', 
              y='avghouseholdsize', 
              title='Household Size Distribution by State',
              labels={'statefips': 'State FIPS', 'avghouseholdsize': 'Average Household Size'},
              color='statefips')
fig4.show()

**SECOND CSV : Cancer analysis**


In [26]:
df_sorted_cancer = df_cancer_reg.sort_values(by='incidencerate', ascending=False).head(20)

In [47]:
# Scatter plot - Median income vs Cancer Incidence Rate
fig4 = px.scatter(df_cancer_reg, 
                  x='medincome', 
                  y='incidencerate', 
                  title='Cancer Incidence Rate vs Median Income',
                  labels={'medincome': 'Median Income', 'incidencerate': 'Cancer Incidence Rate'},
                  color='incidencerate',
                  color_continuous_scale='Viridis')
fig4.show()


In [31]:
fig5 = px.box(df_cancer_reg, 
              x='binnedinc', 
              y='target_deathrate', 
              title='Cancer Death Rate Distribution by Income Level',
              labels={'binnedinc': 'Income Bins', 'target_deathrate': 'Cancer Death Rate'},
              color='binnedinc')
fig5.show()


In [48]:
# Line chart - Average deaths per year vs median age (focused on 0-100)
fig7 = px.line(df_cancer_reg[df_cancer_reg['medianage'] <= 100].sort_values(by='medianage'), 
               x='medianage', 
               y='avgdeathsperyear', 
               title='Average Deaths per Year vs Median Age (0-100)',
               labels={'medianage': 'Median Age', 'avgdeathsperyear': 'Average Deaths per Year'},
               markers=True)
fig7.show()


In [35]:
fig6 = px.histogram(df_cancer_reg, 
                    x='povertypercent', 
                    title='Distribution of Poverty Percentage with KDE',
                    nbins=30, 
                    marginal="violin", 
                    color_discrete_sequence=['orange'])
fig6.show()


**NOW WE MIX BOTH DATAFRAME** (life expectancy and cancer_reg)

i hope your exam went good Isar

In [40]:
# Re-attempt generating the visualizations after the connection issue

import plotly.express as px

# 1. Cancer Incidence Rate vs. Median Income
fig1 = px.scatter(df_cancer_reg, x="medincome", y="incidencerate", 
                  title="Cancer Incidence Rate vs. Median Income",
                  labels={"medincome": "Median Income", "incidencerate": "Cancer Incidence Rate"},
                  trendline="ols")


fig1.show()



In [41]:
fig2 = px.scatter(df_life_expectancy, x="GDP", y="Life expectancy ", 
                  title="Life Expectancy vs. GDP",
                  labels={"GDP": "Gross Domestic Product", "Life expectancy ": "Life Expectancy"},
                  trendline="ols")
fig2.show()

In [42]:
fig3 = px.scatter(df_cancer_reg, x="pctpubliccoverage", y="target_deathrate", 
                  title="Public Health Coverage vs. Cancer Death Rate",
                  labels={"pctpubliccoverage": "Public Health Coverage (%)", "target_deathrate": "Cancer Death Rate"},
                  color="pctprivatecoverage")
fig3.show()


In [43]:
# Alcohol Consumption vs. Life Expectancy
fig4 = px.scatter(df_life_expectancy, x="Alcohol", y="Life expectancy ", 
                  title="Alcohol Consumption vs. Life Expectancy",
                  labels={"Alcohol": "Alcohol Consumption (liters per capita)", "Life expectancy ": "Life Expectancy"},
                  trendline="ols")
fig4.show()

In [44]:
fig5 = px.scatter(df_cancer_reg, x="povertypercent", y="incidencerate", 
                  title="Poverty Rate vs. Cancer Incidence",
                  labels={"povertypercent": "Poverty Rate (%)", "incidencerate": "Cancer Incidence Rate"},
                  trendline="ols")

In [45]:
fig5.show()

let's merge everything

In [37]:
merged_df = df_avg_household_size.merge(df_cancer_reg, left_on='geography', right_on='geography', how='inner')


In [38]:
fig8 = px.scatter(merged_df, 
                  x='avghouseholdsize', 
                  y='incidencerate', 
                  title='Household Size vs Cancer Incidence Rate',
                  labels={'avghouseholdsize': 'Average Household Size', 'incidencerate': 'Cancer Incidence Rate'},
                  color='incidencerate',
                  color_continuous_scale='Plasma')
fig8.show()

In [39]:
fig9 = px.box(merged_df, 
              x='avghouseholdsize', 
              y='target_deathrate', 
              title='Cancer Death Rate vs Household Size',
              labels={'avghouseholdsize': 'Average Household Size', 'target_deathrate': 'Cancer Death Rate'},
              color='avghouseholdsize')
fig9.show()

In [50]:
filtered_cancer_df = df_cancer_reg[df_cancer_reg["medianage"] <= 80]

fig6 = px.scatter(filtered_cancer_df, x="medianage", y="target_deathrate", 
                  title="Cancer Death Rate vs. Median Age (Up to 80 Years)",
                  labels={"medianage": "Median Age", "target_deathrate": "Cancer Death Rate"},
                  trendline="ols")

# Show the figure
fig6.show()

